```sql
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.
```

# **INTRODUCTIONS**


# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/conversions.ipynb",
        "src/utils/schemas.ipynb",
        "src/utils/protos.ipynb",
    ]
}
```

# **TABLES**

---


### STAGINGS

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]'))
CREATE TABLE [VWSSTAGE].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN](
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [VERSION] [VARCHAR](100) NULL,
  [DATE_OF_REPORT] [VARCHAR](100) NULL,
  [DATE_OF_STATISTICS_WEEK_START] [VARCHAR](100) NULL,
	[DATE_OF_STATISTICS_WEEK_END] [VARCHAR](100) NULL,
	[BIRTH_COHORT] [VARCHAR](100) NULL,
  [AGE_GROUP] [VARCHAR](100) NULL,
  [POPULATION_AGE_GROUP] [VARCHAR](100) NULL,
  [VACCINE_SERIE] [VARCHAR](100) NULL,
  [RECEIVED] [VARCHAR](100) NULL,
  [COVERAGE] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

### INTERMEDIATES

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSINTER].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]'))
CREATE TABLE [VWSINTER].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN](
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [DATE_OF_REPORT] [DATETIME] NULL,
  [DATE_OF_STATISTICS_WEEK_START] [DATETIME] NULL,
	[DATE_OF_STATISTICS_WEEK_END] [DATETIME] NULL,
	[BIRTH_COHORT] [VARCHAR](100) NULL,
  [AGE_GROUP] [VARCHAR](100) NULL,
  [POPULATION_AGE_GROUP] [BIGINT] NULL,
  [VACCINE_SERIE] [VARCHAR](100) NULL,
  [RECEIVED] [BIGINT] NULL,
  [COVERAGE] [DECIMAL](16,2) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

### DESTINATIONS

In [ ]:
SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSDEST].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]'))
CREATE TABLE [VWSDEST].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN](
	[ID] [BIGINT] PRIMARY KEY IDENTITY(1,1),
  [DATE_OF_REPORT] DATETIME NULL,
  [DATE_OF_STATISTICS_WEEK_START] DATETIME NULL,
  [DATE_OF_STATISTICS_WEEK_END] DATETIME NULL,
  [BIRTH_COHORT] VARCHAR(100) NULL,
  [AGE_GROUP] VARCHAR(100) NULL,
  [AGE_GROUP_POPULATION] BIGINT NULL,
  [AGE_GROUP_POPULATION_PERCENTAGE] DECIMAL(16,2) NULL,
  [VACCINE_SERIE] [VARCHAR](100) NULL,
  [RECEIVED] [BIGINT] NULL,
  [COVERAGE] [DECIMAL](16,2) NULL,
  [DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE()
);
GO

-- 2) CREATE INDEX(ES).....
DROP INDEX IF EXISTS [NCIX_DLI_DOS_AG_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN] 
  ON [VWSDEST].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]
GO

CREATE NONCLUSTERED INDEX [NCIX_DLI_DOS_AG_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN] 
  ON [VWSDEST].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN] (
    [DATE_LAST_INSERTED], 
    [DATE_OF_STATISTICS_WEEK_START], 
    [DATE_OF_STATISTICS_WEEK_END], 
    [AGE_GROUP]
  );
GO

# **STORED PROCEDURES**

---


### STAGING &rarr; INTERMEDIATE MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT. 
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_INTER_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]
AS
BEGIN
    INSERT INTO [VWSINTER].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN] (
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [DATE_OF_STATISTICS_WEEK_END],
        [BIRTH_COHORT],
        [AGE_GROUP],
        [POPULATION_AGE_GROUP],
        [VACCINE_SERIE],
        [RECEIVED],
        [COVERAGE]
    )
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [DATE_OF_STATISTICS_WEEK_END],
        [BIRTH_COHORT],
        [AGE_GROUP],
        [POPULATION_AGE_GROUP],
        [VACCINE_SERIE],
        [RECEIVED],
        [COVERAGE]
    FROM [VWSSTAGE].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]);
END;
GO

### INTERMEDIATE &rarr; DESTINATION MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER  PROCEDURE [DBO].[SP_DEST_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]
AS
BEGIN
-- APPLY FILTERS ON MAIN INPUT TABLE AND PERFORM TRANSFORMATIONS
WITH LAST_CTE AS (
    SELECT
        [DATE_OF_REPORT],
        [DATE_OF_STATISTICS_WEEK_START],
        [DATE_OF_STATISTICS_WEEK_END],
        [BIRTH_COHORT],
        [AGE_GROUP],
        [POPULATION_AGE_GROUP],
        [VACCINE_SERIE],
        [RECEIVED],
        [COVERAGE]
    FROM [VWSINTER].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN]
    WHERE [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSINTER].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN])
        AND LOWER([BIRTH_COHORT]) != 'unknown'
)
INSERT INTO [VWSDEST].[CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN](
    [DATE_OF_REPORT],
    [DATE_OF_STATISTICS_WEEK_START],
    [DATE_OF_STATISTICS_WEEK_END],
    [AGE_GROUP],
    [VACCINE_SERIE],
    [RECEIVED],
    [COVERAGE]
)
SELECT
    T.[DATE_OF_REPORT],
    T.[DATE_OF_STATISTICS_WEEK_START],
    T.[DATE_OF_STATISTICS_WEEK_END],
    T.[AGE_GROUP],
    T.[VACCINE_SERIE],
    T.[RECEIVED],
    T.[COVERAGE]
    FROM LAST_CTE AS T
    --TAKE ONLY THE LAST VALUE
    WHERE T.[DATE_OF_STATISTICS_WEEK_START] = (SELECT MAX([DATE_OF_STATISTICS_WEEK_START]) FROM LAST_CTE)
END
GO

# **ORCHESTRATOR CONFIGURATIONS**

---

> WORKFLOWS

In [ ]:
-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 12 * * WED', -- AT 02:00 PM, ON WEDNESDAY
    @active = @is_active;
GO

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50),
        @delimiter_type VARCHAR(50),
        @source_type VARCHAR(50);



-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#') --https://data.rivm.nl/data/vws/covid-19/VWS_COVID-19_campagne_vaccinatiegraad_per_week_geboortejaar.csv
    WHEN 'production' THEN 'datafiles/RIVM/VWS_COVID-19_campagne_vaccinatiegraad_per_week_geboortejaar.csv'
    WHEN 'acceptance' THEN 'datafiles/RIVM/VWS_COVID-19_campagne_vaccinatiegraad_per_week_geboortejaar.csv'
    ELSE 'datafiles/RIVM/VWS_COVID-19_campagne_vaccinatiegraad_per_week_geboortejaar.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'AzureBlob' THEN 'Web'
    WHEN 'AzureBlob' THEN 'Web'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

SET @delimiter_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'SemiColon'
    WHEN 'acceptance' THEN 'SemiColon'
    ELSE 'SemiColon'
END;

SET @source_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'CsvFile'
    WHEN 'acceptance' THEN 'CsvFile'
    ELSE 'CsvFile'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = CONCAT('SOURCE_', @workflow_name);
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'Version|Date_of_report|Date_of_statistics_week_start|Date_of_statistics_week_end|Birth_cohort|Age_group|Population_age_group|Vaccine_serie|Received|Coverage',
    @target_columns = 'VERSION|DATE_OF_REPORT|DATE_OF_STATISTICS_WEEK_START|DATE_OF_STATISTICS_WEEK_END|BIRTH_COHORT|AGE_GROUP|POPULATION_AGE_GROUP|VACCINE_SERIE|RECEIVED|COVERAGE',
    @target_name = @target_name,
    @source_type = @source_type,
    @location_type = @location_type,
    @delimiter_type = @delimiter_type,
    @security_profile= @security_profile;

-- 2.3) UPSERT STORED PROCEDURE SOURCE(S): STAGE TO INTER......
SET @source = 'DBO.SP_INTER_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN';
SET @source_name = 'SOURCE_SP_INTER_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN';
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 2.4) UPSERT STORED PROCEDURE SOURCE(S): INTER TO DEST.....
SET @source = 'dbo.SP_DEST_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN'
SET @source_name = 'SOURCE_SP_DEST_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN'
SET @source_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST'); 

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = null,
    @target_columns = null,
    @target_name = null,
    @source_type = 'StoredProcedure',
    @location_type = 'N/A',
    @delimiter_type = 'N/A',
    @security_profile= @security_profile;

-- 3) UPSERT PROCESS(ES).....
DECLARE @process_id INT,
        @process_name NVARCHAR(256),
        @process_description NVARCHAR(256),
        @process_source_name VARCHAR(256);

-- 3.1) UPSERT TABLE PROCESS(ES): STAGING......
SET @process_name = CONCAT('PROCESS_', @workflow_name, '_TO_STAGE');
SET @process_description = CONCAT('LOAD: ', @workflow_name);
SET @process_source_name = CONCAT('SOURCE_', @workflow_name);

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.2) UPSERT STORED PROCEDURE PROCESS(S): STAGE TO INTER......
SET @process_name = CONCAT('PROCESS_', @workflow_name, '_STAGE_TO_INTER');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM STAGE TO INTER');
SET @process_source_name = 'SOURCE_SP_INTER_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 3.3) UPSERT STORED PROCEDURE PROCESS(S): INTER TO DEST......
SET @process_name = CONCAT('PROCESS_', @workflow_name, '_INTER_TO_DEST');
SET @process_description = CONCAT('MAP: ', @workflow_name, ' FROM INTER TO DEST');
SET @process_source_name = 'SOURCE_SP_DEST_CIMS_VACCINATED_PER_AGE_GROUP_CAMPAIGN';

SELECT TOP(1)
    @process_id = processes.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[PROCESSES] processes
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[ID] = processes.[WORKFLOW_ID]
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_PROCESSES] v_processes ON v_processes.[PROCESS_ID] = processes.[ID]
WHERE v_processes.[PROCESS_NAME] = @process_name 
    AND v_workflows.[NAME] = @workflow_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_PROCESS]
    @id = @process_id,
    @process_name = @process_name,
    @description = @process_description,
    @source_name = @process_source_name,
    @schedule = '* * * * *',
    @workflow_name = @workflow_name,
    @active = 1;

-- 4) UPSERT DEPENDENC(Y)(IES).....
DECLARE @dependency_id INT,
        @dependency_name NVARCHAR(256),
        @dependency_description NVARCHAR(256),
        @dependency_dataflow_name NVARCHAR(256),
        @dependency_process_name NVARCHAR(256);

-- 4.1) UPSERT STAGING TO STORED PROCEDURE DEPENDENC(Y)(IES).....
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_STAGE_TO_INTER');
SET @dependency_dataflow_name = CONCAT('PROCESS_', @workflow_name, '_STAGE_TO_INTER');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name, '_TO_STAGE');
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;

-- 4.2) UPSERT STORED PROCEDURE TO STORED PROCEDURE DEPENDENC(Y)(IES)......
SET @dependency_name = CONCAT('DEPENDENCY_', @workflow_name,'_INTER_TO_DEST');
SET @dependency_dataflow_name = CONCAT('PROCESS_', @workflow_name, '_INTER_TO_DEST');
SET @dependency_process_name = CONCAT('PROCESS_', @workflow_name, '_STAGE_TO_INTER');
SET @dependency_description = CONCAT('TRIGGER ', @dependency_dataflow_name,' AFTER ', @dependency_process_name ,' HAS FINISHED');

SELECT TOP(1)
    @dependency_id = dependencies.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[DEPENDENCIES] dependencies
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_DEPENDENCIES] v_dependencies ON v_dependencies.[ID] = dependencies.[ID]
WHERE dependencies.[NAME] = @dependency_name
    AND v_dependencies.[WORKFLOW_NAME] = @workflow_name
    AND v_dependencies.[DEPENDENCY_NAME] = @dependency_process_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_DEPENDENCY]
    @id = @dependency_id,
    @dataflow_name = @dependency_dataflow_name,
    @dataflowtype_id = 2,
    @dependency_name = @dependency_process_name,
    @dependencytype_id = 2,
    @workflow_name = @workflow_name,
    @name = @dependency_name,
    @description = @dependency_description,
    @active = 1;
GO